In [1]:
from utility import *

Specify the Qubit Hamiltonian of a molecule by name, internuclear distances, and the basis set.
Here, we show all the relevant terms of H2. 

In [2]:
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g')

# This is due to a current tequila bug that fires when constant is invovled. We should be able to get rid of this line by July 1
constant = h2.constant
h2 = h2 - QubitOperator(term=[], coefficient=constant)
print(h2)

(0.04919764587136755+0j) [X0 Z1 X2] +
(0.04919764587136755+0j) [X0 Z1 X2 Z3] +
(0.04919764587136755+0j) [Y0 Z1 Y2] +
(0.04919764587136755+0j) [Y0 Z1 Y2 Z3] +
(0.13716572937099497+0j) [Z0] +
(0.13716572937099492+0j) [Z0 Z1] +
(0.15542669077992832+0j) [Z0 Z1 Z2] +
(0.15542669077992832+0j) [Z0 Z1 Z2 Z3] +
(0.10622904490856075+0j) [Z0 Z2] +
(0.10622904490856075+0j) [Z0 Z2 Z3] +
(0.15660062488237947+0j) [Z1] +
(-0.13036292057109117+0j) [Z1 Z2 Z3] +
(0.16326768673564346+0j) [Z1 Z3] +
(-0.13036292057109117+0j) [Z2]


However, not all terms are measurable at once in a quantum computer. Since the quantum computer measures qubit by qubit, each measurable fragments must all have the same terms (X, Y or Z) on each qubit. 

Notice below that all 3 fragments have the same terms on all qubits. They are qubit-wise commuting. 

In [3]:
qwc_list = get_qwc_group(h2)
print('1st fragments: \n {}\n'.format(qwc_list[0]))
print('2nd fragments:\n {}\n'.format(qwc_list[1]))
print('3rd fragments:\n {}\n'.format(qwc_list[2]))

1st fragments: 
 (0.04919764587136755+0j) [Y0 Z1 Y2] +
(0.04919764587136755+0j) [Y0 Z1 Y2 Z3] +
(0.15660062488237947+0j) [Z1] +
(0.16326768673564346+0j) [Z1 Z3]

2nd fragments:
 (0.04919764587136755+0j) [X0 Z1 X2] +
(0.04919764587136755+0j) [X0 Z1 X2 Z3]

3rd fragments:
 (0.13716572937099497+0j) [Z0] +
(0.13716572937099492+0j) [Z0 Z1] +
(0.15542669077992832+0j) [Z0 Z1 Z2] +
(0.15542669077992832+0j) [Z0 Z1 Z2 Z3] +
(0.10622904490856075+0j) [Z0 Z2] +
(0.10622904490856075+0j) [Z0 Z2 Z3] +
(-0.13036292057109117+0j) [Z1 Z2 Z3] +
(-0.13036292057109117+0j) [Z2]



Obtain measurable parts by partitioning into mutually commuting fragments. 

In [4]:
comm_groups = get_commuting_group(h2)
print('Number of mutually commuting parts: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting parts: 2
The first commuting group
(0.04919764587136755+0j) [X0 Z1 X2] +
(0.04919764587136755+0j) [X0 Z1 X2 Z3] +
(0.04919764587136755+0j) [Y0 Z1 Y2] +
(0.04919764587136755+0j) [Y0 Z1 Y2 Z3] +
(0.15542669077992832+0j) [Z0 Z1 Z2] +
(0.15542669077992832+0j) [Z0 Z1 Z2 Z3] +
(0.10622904490856075+0j) [Z0 Z2] +
(0.10622904490856075+0j) [Z0 Z2 Z3] +
(0.15660062488237947+0j) [Z1] +
(0.16326768673564346+0j) [Z1 Z3]


To see each part is indeed measurable, one can construct the unitary operator that rotates all terms into qubit-wise commuting terms.

In [5]:
u = get_qwc_unitary(comm_groups[1])
print('This is unitary. U * U^+ = I ')
print(u * u)

This is unitary. U * U^+ = I 
(0.9999999999999996+0j) []


The qubit-wise commuting form of the first mutually commuting group

In [6]:
print(u * comm_groups[1] * u)

(0.1062290449085607+0j) [X0] +
(0.1554266907799282+0j) [X0 X1] +
(-0.04919764587136754+0j) [X0 X1 Z2] +
(-0.04919764587136754+0j) [X0 X1 Z2 X3] +
(0.1554266907799282+0j) [X0 X1 X3] +
(0.1062290449085607+0j) [X0 X3] +
(0.1566006248823793+0j) [X1] +
(0.04919764587136754+0j) [X1 Z2] +
(0.04919764587136754+0j) [X1 Z2 X3] +
(0.16326768673564332+0j) [X1 X3]


Alternatively, the qubit-tapering technique can exploit the symmetry in the Hamiltonian and find a smaller effective Hamitlonian. 

In [12]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2))) 
I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])

h2_matrix = -0.20344315 * I + 0.19679058 * X - 0.53505729 * Z
eigvals, _ = np.linalg.eigh(h2_matrix)
print("\nThe eigenvalues including ground state energy: \n {}".format(eigvals + constant))

The effective Hamiltonian:
 (-0.20344315975895527+0j) [] +
(0.1967905834854702+0j) [X0] +
(-0.5350572998841723+0j) [Z0]

The eigenvalues including that of ground state: 
 [-1.10115031+0.j  0.03904763+0.j]
